Tutorial SVM Python

Por:

Jonathan Navarro

Antonio Piedra

Dominic Tarassenko


Modificado por

Marco Ferraro

Gabriel Revillat

Steven Nuñez


Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, svm
import sklearn.metrics as skm

Primero, se cargan y preparan los datos

Se puede hacer lematizacion adicional tokenizando los datos de forma manual, y luego usando nltk, tomando en consideración que después de tiene que volver a rejuntar los textos. Para más información sobre preprocesado de lenguaje natural puede revisar este link: https://www.nltk.org/

In [2]:
nombreDoc = "MoviesUSAGermany.csv"
nombreDocTest = "MovieUSAGermanyTesting.csv"
nombreEtiquetaX = "title"
nombreEtiquetaY = "country"
np.random.seed(500)
Corpus = pd.read_csv(nombreDoc,encoding='latin-1')
Testing = pd.read_csv(nombreDocTest,encoding='latin-1')

In [3]:
# Elimina las filas vacias
Corpus[nombreEtiquetaX].dropna(inplace=True)
# Pasa todo a minuscula
Corpus[nombreEtiquetaX] = [entry.lower() for entry in Corpus[nombreEtiquetaX]]

In [4]:
Corpus

,country,title
0,Germany,abikalypse
1,USA,abilene
2,USA,ablaze
3,USA,able danger
4,USA,able edwards
...,...,...
11367,USA,il primo ribelle
11368,USA,il primo uomo diventato donna
11369,USA,il principe cerca moglie
11370,USA,il principe consorte


In [5]:
# Elimina las filas vacias
Testing[nombreEtiquetaX].dropna(inplace=True)
# Pasa todo a minuscula
Testing[nombreEtiquetaX] = [entry.lower() for entry in Corpus[nombreEtiquetaX]]

ValueError: Length of values (11372) does not match length of index (5062)

In [6]:
Testing

,country,title
0,USA,Il principe degli attori
1,USA,Il principe del circo
2,USA,Il principe delle donne
3,USA,Il principe delle maree
4,USA,Il principe delle volpi
...,...,...
5057,USA,Missione segreta
5058,USA,Missione suicida
5059,USA,Missione suicidio
5060,USA,Mississippi


In [7]:
#Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus[nombreEtiquetaX],Corpus[nombreEtiquetaY],test_size=0.2)

In [8]:
Train_X = Corpus[nombreEtiquetaX]
Train_Y = Corpus[nombreEtiquetaY]
Test_X = Testing[nombreEtiquetaX]
Test_Y = Testing[nombreEtiquetaY]

Encoding

Convierte datos categoricos de tipo string en valores numericos que el modelo pueda entender

In [9]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
etiqueta0, etiqueta1 = Encoder.inverse_transform([0])[0], Encoder.inverse_transform([1])[0]
print(etiqueta0, etiqueta1)
Test_Y = Encoder.fit_transform(Test_Y)

Germany USA


Vectorization

Se usa TfidfTransformer, equivale a hacer un bag of words y luego aplicarle tf-idf. Básicamente en lugar de usar el conteo de palabras de forma directa, también considera qué tan común es a lo largo de los documentos, entonces una palabra que está en muchos documentos da menos información para distinguirlos entre ellos.

Se puede usar CountVectorizer si se quiere únicamente utilizar Bag of Words

Más información
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
https://en.wikipedia.org/wiki/Tf%E2%80%93idf

In [10]:
Tfidf_vect = TfidfVectorizer(ngram_range=(1,2))
Tfidf_vect.fit(Corpus[nombreEtiquetaX])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [11]:
print(Tfidf_vect.vocabulary_)

{'abikalypse': 175, 'abilene': 176, 'ablaze': 182, 'able': 183, 'danger': 7920, 'able danger': 184, 'edwards': 10873, 'able edwards': 185, 'abner': 186, 'the': 22950, 'invisible': 16963, 'dog': 10060, 'abner the': 187, 'the invisible': 23157, 'invisible dog': 16964, 'abnormal': 188, 'attraction': 2336, 'abnormal attraction': 189, 'abominable': 190, 'about': 192, 'girl': 13941, 'about girl': 198, 'alex': 695, 'about alex': 193, 'cherry': 5906, 'about cherry': 194, 'face': 11617, 'about face': 196, 'fifty': 12144, 'about fifty': 197, 'last': 17544, 'night': 19178, 'about last': 200, 'last night': 17549, 'scout': 21650, 'about scout': 203, 'above': 208, 'beyond': 3444, 'above beyond': 209, 'ground': 14648, 'segreti': 21727, 'sepolti': 21837, 'above ground': 210, 'ground segreti': 14649, 'segreti sepolti': 21733, 'suspicion': 22738, 'above suspicion': 211, 'rim': 21163, 'above the': 212, 'the rim': 23267, 'shadows': 21931, 'the shadows': 23295, 'abrafaxe': 213, 'pirati': 20386, 'dei': 8464

Entrenar el SVM

Los tipos de kernel soportados son 'linear', 'poly', 'rbf', 'sigmoid'
Gamma solo se usa cuando se usa rbf, poly o sigmoid, para el resto es ignorado.
Degree solo se usa para poly y es ignorado para los demas

linear suele ser mejor para clasificar textos

Más información:
https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

In [18]:
# Ligar el dataset con el clasificador
SVM = svm.SVC(C=1, kernel='rbf', degree=3, gamma='scale')
SVM.fit(Train_X_Tfidf,Train_Y)
# Hacer las predicciones con el conjunto de prueba
predictions_SVM = SVM.predict(Test_X_Tfidf)

Se miden las estadísticas importantes con las pruebas

Estas no son las únicas que se pueden hacer, una lista exhaustiva se puede encontrar en el siguiente link:
https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics

In [19]:
print("SVM Accuracy Score -> ", skm.accuracy_score(Test_Y, predictions_SVM)*100)
print("SVM Recall ", etiqueta0, skm.recall_score(Test_Y, predictions_SVM, pos_label=0))
print("SVM Recall ", etiqueta1, skm.recall_score(Test_Y, predictions_SVM, pos_label=1))
print("SVM Precision  ", etiqueta0, skm.precision_score(Test_Y, predictions_SVM,pos_label=0))
print("SVM Precision  ", etiqueta1, skm.precision_score(Test_Y, predictions_SVM,pos_label=1))
print("SVM F  ", etiqueta0, skm.f1_score(Test_Y, predictions_SVM,pos_label=0))
print("SVM F  ", etiqueta1, skm.f1_score(Test_Y, predictions_SVM,pos_label=1))
print("SVM Confusion Table: \n", skm.confusion_matrix(Test_Y, predictions_SVM))

SVM Accuracy Score ->  93.71789806400632
SVM Recall  Germany 0.0893371757925072
SVM Recall  USA 0.999575821845175
SVM Precision   Germany 0.9393939393939394
SVM Precision   USA 0.9371644462119706
SVM F   Germany 0.16315789473684209
SVM F   USA 0.9673645320197044
SVM Confusion Table: 
 [[  31  316]
 [   2 4713]]
